### Classes & Metaclasses



In [1]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    # repr = represent 
    # the *{!r} is the repr ... 
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    # zip() takes an iterable and outputs an iterable... 
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    
    # you have to look at the documentation... len is pre-defined in some sense
    # len is thematically in the documentation, to evoke some sense of size
    # the pattern is to think of the higher level function acting on self... hence len(self.coeffs) 
    def __len__(self):
        return len(self.coeffs)
    
    def __call__(self):
        pass

In [2]:
p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

p1 + p2

Polynomial(*(4, 6, 6))

In [3]:
len(p1) 

3

### Metaclasses 



In [4]:
# File 1 - library.py
# This is the infrastructure side... 
class Base:
    def food(self):
        return 'foo'

In [13]:
# File2 - user.py
# This is the user developer side... assert that this exists in the library side before you execute. 
# This way, you can catch the error before you go into production. 
assert hasattr(Base, 'foo'), "you broke it, you fool!"

class Derived(Base):
    def bar(self):
        return self.foo

In [6]:
# File 1 - library.py
# now the shoe is on the other foot... 
# and now the library side is using somethin that's made on the user side??? not sure (RETURN TO THIS)


class Base:
    def foo(self):
        return self.bar()
    

In [7]:
# File2 - user.py

assert hasattr(Base, 'foo'), "you broke it, you fool!"

class Derived(Base):
    def bar(self):
        return 'bar'

In [9]:
Derived.bar

<function __main__.Derived.bar>

In [10]:
def _():
    class Base:
        pass

from dis import dis

In [11]:
# dis = dissembles

dis(_) # LOAD_BUILD_CLASS

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x1064e39c0, file "<ipython-input-10-f0669648f7a8>", line 2>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


## Great introduction to decorators
Core idea is to think about wrappers. If you have a function that returns a function... There is an important concept here as a closure object duality but we won't go into this. 

- Fundamentally, you can wrap wide swathes of functions without having to re-write a lot of user code or have a lot of churn in the library code. 
- `*args and  **kwargs` --> can take arbitrary parameter specs and forward it along in any fashion


In [2]:
from time import time

# you can define functions anywhere, at run-time too
def timer(func): 
    def f(*args, **kwargs): 
        before = time() # wrap the function with behavior before and after
        rv = func(*args, **kwargs) # rv = return value
        after = time() 
        print('elapsed', after - before) 
        return rv
    return f

@timer
def add(x, y=10): 
    return x + y
# the decorator replaces the need for this line below
# a decorator is syntax that's equal to function = wrapper(function)
# you can add in this functionality without needing to re-write this 
# extra user code...! now you can write it at the top.
# add = timer(add)

@timer
def sub(x, y=10): 
    return x - y
# sub = timer(sub)

print('add(10)', add(10))
print('add(20,30)', add(20,30))

elapsed 9.5367431640625e-07
add(10) 20
elapsed 9.5367431640625e-07
add(20,30) 50
